# Tutorial: Fermi GBM Spectral Catalog Data

Adapted from "GBM Spectral Catalog Files" in the Fermi GBM Data Tools documentation and "Fermi GBM Spectral Fits Data" in the Fermi Gamma-ray Data Tools documentation. 

GBM provides standard spectral fits for each GRB triggered on-board. These spectral fits are hosted in FITS format at HEASARC as “SCat” (Spectral Catalog) files. The SCat files contain the spectral fit data, including fit parameters, uncertainties, fluxes/fluences, fit statistic, and covariance. The files also contain metadata about the detectors and data that were used in the fit. All of this information can be easily accessed using the `Scat` class, although currently (as of summer 2024), SCat files for GRBs after summer 2018 do not yet exist. 

In this tutorial, we will go through an example of how to use the Fermi Gamma-Ray Data Tools (GDT) toolkit as well as the core GDT toolkit to download, access, and manipulate SCat data from the Fermi Gamma-ray Burst Monitor. 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import gdt.missions.fermi 
import gdt.core

First, we need to download and read our SCat FITS file. We can download SCat files using the `TriggerFtp` class from the GDT Fermi toolkit, although this only lets us download all the catalog files at once (burst catalog or BCat, trigger catalog or TCat, and all SCat files) and not pick individual files. After downloading the files, we can then use the `Scat` class to open and read the specific SCat file that we want. 

For this example, we will be using an SCat file containing the Comptonized fit of the fluence spectrum (labeled "flnc_comp"), which means that the Comptonized function is being used to fit the spectrum data from the entire duration of the burst (a.k.a. the fluence spectrum). Other fitting functions that the Spectral Catalog uses are the Power Law ("plaw") function, the Band ("band") function, and the Smoothly Broken Power Law ("sbpl") function. The alternative spectrum to the fluence spectrum is known as the peak flux ("pflx") spectrum, which only considers data from the brightest portion of the brust. We will use event **170817529** as our example event for the rest of this tutorial. 

In [34]:
# import classes needed to download data from triggered events
from gdt.missions.fermi.gbm.finders import TriggerFtp
from gdt.core import data_path

# import the Scat class
from gdt.missions.fermi.gbm.scat import Scat

# initialize TriggerFtp trigger data finder to pull data from event 170817529
scat_finder = TriggerFtp('170817529')

# download all the catalog files associated with the trigger (burst catalog/bcat, spectral catalog/scat, trigger catalog/tcat)
scat_finder.get_cat_files(download_dir=data_path.joinpath('170817529'))

# read an scat file for the comptonized fit of the fluence spectrum
scat = Scat.open(data_path.joinpath('170817529/glg_scat_all_bn170817529_flnc_comp_v00.fit'))
print(scat)

Output()

Output()

Output()

Output()

Output()

...

Output()

Output()

Output()

Output()

<Scat: glg_scat_all_bn170817529_flnc_comp_v00.fit;
 4 detectors; 1 fits>


After opening our file, we can quickly determine the number of detectors used in the spectral fit(s) and the number of spectral fits provided in the file using the `num_detectors` and `num_fits` attributes. 

In [16]:
# print how many detectors were used in the spectral fits
print('Number of detectors used: {}'.format(scat.num_detectors))

# print how many spectral fits the file contains
print('Number of spectral fits: {}'.format(scat.num_fits))

Number of detectors used: 4
Number of spectral fits: 1


You can also access the list of model fits by the `model_fits` attribute.

In [17]:
# return the list of model fits provided in the file
scat.model_fits

[<GbmModelFit: Comptonized, Epeak>]

Using the `model_fits` attribute, you can also access each individual provided fit as a `GbmModelFit` object. The `GbmModelFit` class contains the all of the relevant information for a fit. Let’s look at a few properties contained in this class, such as the time range, statistic, degrees of freedom, model name, and model parameters of the fit:

In [39]:
# access the comptonized model fit
one_fit = scat.model_fits[0]

# the time range of the model fit
print('Fit time range: {0} - {1}'.format(*one_fit.time_range))

# the name of the statistic that was used for the fit
print('Fit statistic: {}'.format(one_fit.stat_name))

# the value of the fit statistic and the degrees of freedom of the fit
print('Stat/DoF: {0:.2f}/{1}'.format(one_fit.stat_value, one_fit.dof))

# the name of the model
print('Model name: {}'.format(one_fit.name))

# the names of the parameters used in the fit
print('Parameter names: {}'.format(one_fit.parameter_list()))

Fit time range: -0.192 - 0.064
Fit statistic: Castor C-STAT
Stat/DoF: 516.50/479
Model name: Comptonized, Epeak
Parameter names: ['Amplitude', 'Epeak', 'Index', 'Pivot E =fix']


The actual fit data are stored as `Parameter` objects, which group together the fit value, uncertainty, and metadata such as the parameter name and units. If we want to quickly see the fit information, we can simply print the `GbmModelFit` object.

In [19]:
# quickly view the parameter values for the comptonized fit
print(one_fit)

Comptonized, Epeak
   Amplitude: 0.03 +/- 0.02
   Epeak: 215.09 +/- 54.22
   Index: 0.14 +/- 0.59
   Pivot E =fix: 1.00e+02 +/- 0.00e+00


Otherwise, we can directly access the parameter fit info, such as parameter names, values, uncertainties, and 1-sigma errors by the following attributes and methods of the `Parameter` class.

In [20]:
# access the 'epeak' parameter from the comptonized fit
epeak = one_fit.parameters[1]

# print the name of the accessed parameter
print('Name: {}'.format(epeak.name))

# print the value of the epeak parameter
print('Value: {}'.format(epeak.value))

# print the upper and lower 1-sigma uncertainty for the epeak parameter
print('1-sigma uncert: {}'.format(epeak.uncertainty))

# print the 1-sigma range of the epeak parameter
print('1-sigma range: {}'.format(epeak.one_sigma_range()))

Name: Epeak
Value: 215.09434509277344
1-sigma uncert: (54.219116, 54.219116)
1-sigma range: (160.87522888183594, 269.31346130371094)


For each fit, the photon and energy fluxes and fluences are recorded and stored as special `Parameter` objects as well. They can be accessed by attributes such as `GbmModelFit.photon_flux` and `GbmModelFit.energy_fluence`.

In [21]:
# print the photon flux associated with the comptonized fit
print(one_fit.photon_flux)

# print the energy fluence associated with the comptonized fit
print(one_fit.energy_fluence)

Photon Flux: 2.81 +/- 0.44 ph/cm^2/s
Energy Fluence: 1.40e-07 +/- 3.03e-08 erg/cm^2


As for the detector metadata used in the fits, they are stored in `GbmDetectorData` objects and can be accessed using the `Scat.detectors` attribute.

In [23]:
# print the detectors used in the analysis contained in the scat file
print(scat.detectors)

[<GbmDetectorData: BGO_00; TTE;
 time range: (-0.192, 0.064) s;
 energy range: (284.65, 40108.0) keV>, <GbmDetectorData: NAI_01; TTE;
 time range: (-0.192, 0.064) s;
 energy range: (8.501, 903.45) keV>, <GbmDetectorData: NAI_02; TTE;
 time range: (-0.192, 0.064) s;
 energy range: (9.119, 902.57) keV>, <GbmDetectorData: NAI_05; TTE;
 time range: (-0.192, 0.064) s;
 energy range: (8.698, 909.58) keV>]


We can access various properties stored in `GbmDetectorData` objects, such as the detector names, their energy and energy channel ranges, and whether or not data from each detector was used in the fit calculations (a.k.a. whether or not the detector was active), using the class's various attributes. 

In [25]:
# print the name, energy range, energy channel range, and active status (if the detector was used in the fit) for each detector
for det in scat.detectors:
    print('{0}: energy range: {1} keV, # channel range: {2}, active: {3}'.format(det.detector, det.energy_range, det.channel_range, det.active))


BGO_00: energy range: (284.65, 40108.0) keV, # channel range: (3, 124), active: True
NAI_01: energy range: (8.501, 903.45) keV, # channel range: (5, 124), active: True
NAI_02: energy range: (9.119, 902.57) keV, # channel range: (5, 124), active: True
NAI_05: energy range: (8.698, 909.58) keV, # channel range: (5, 124), active: True


In addition to these properties, the deconvolved photon counts, errors, and detector-convolved photon model are stored as well. These can be used for making model count rate plots. For example:

In [13]:
# the detector-convolved photon model values for detector n1
scat.detectors[1].photon_model

array([1.96157284e-02, 1.99605655e-02, 2.02203058e-02, 2.04198491e-02,
       2.05748081e-02, 2.06954200e-02, 2.07886994e-02, 2.08651833e-02,
       2.09236853e-02, 2.09595580e-02, 2.09755581e-02, 2.09734701e-02,
       2.09545959e-02, 2.09202804e-02, 2.08671484e-02, 2.07942519e-02,
       2.07068976e-02, 2.06062403e-02, 2.04924233e-02, 2.03650203e-02,
       2.02121083e-02, 2.00296603e-02, 1.98248476e-02, 1.96060352e-02,
       1.94124747e-02, 1.92341618e-02, 1.90395378e-02, 1.88379139e-02,
       1.86298881e-02, 1.84038971e-02, 1.81599278e-02, 1.79103129e-02,
       1.76558699e-02, 1.73842870e-02, 1.70959160e-02, 1.68044139e-02,
       1.65107511e-02, 1.62022281e-02, 1.58796720e-02, 1.55573804e-02,
       1.52229378e-02, 1.48775103e-02, 1.45354364e-02, 1.41847273e-02,
       1.38267530e-02, 1.34753715e-02, 1.31187867e-02, 1.27577381e-02,
       1.24046309e-02, 1.20481234e-02, 1.16888676e-02, 1.13384118e-02,
       1.09861176e-02, 1.06325680e-02, 1.02785071e-02, 9.92442667e-03,
      

Congrats, you've completed the lesson on Spectral Catalog data!